In [272]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import json
import pathlib
import pickle
import os
current_path = os.getcwd()
current_path

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('float_format', '{:f}'.format)

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

from datetime import datetime, timedelta
import statistics
import time

from geoband.API import *
import geopandas as gpd
import folium
from folium.plugins import FastMarkerCluster, MarkerCluster
import geoplot as gplt
import geoplot.crs as gcrs
import imageio
import mapclassify as mc

import random
from functools import reduce
from collections import defaultdict

from IPython.display import display
from tqdm.notebook import tqdm
from tqdm import tqdm, tqdm_notebook

import sklearn.cluster
import tensorflow as tf
import pydeck as pdk
import cufflinks as cf 
cf.go_offline(connected=True)
cf.set_config_file(theme='polar')

import shapely
from shapely import wkt
from shapely.geometry import Polygon, Point, shape

# 최적화 solver
from mip import Model, xsum, maximize, BINARY  

# font 
import matplotlib.font_manager as font_manager
path = current_path+'/NanumBarunGothic.ttf'
fontprop = font_manager.FontProperties(fname=path)

font_dirs = [current_path, ]
font_files = font_manager.findSystemFonts(fontpaths=font_dirs)
font_list = font_manager.createFontList(font_files)
font_manager.fontManager.ttflist.extend(font_list)
plt.rcParams["font.family"] = 'NanumGothic'
mpl.rcParams['font.family'] = 'NanumBarunGothic'

In [2]:
# Data Load 
df_01 = pd.read_csv(current_path+'/input/1.수원시_버스정류장.csv')
df_02 = pd.read_csv(current_path+'/input/2.수원시_버스정류장별_승하차이력(1).csv')
df_03 = pd.read_csv(current_path+'/input/3.수원시_버스정류장별_승하차이력(2).csv')
df_04 = pd.read_csv(current_path+'/input/4.수원시_버스정류장별_승하차이력(3).csv')
df_05 = pd.read_csv(current_path+'/input/5.수원시_버스정류장별_승하차이력(4).csv')
df_06 = pd.read_csv(current_path+'/input/6.수원시_버스정류장별_승하차이력(5).csv')
df_07 = pd.read_csv(current_path+'/input/7.수원시_버스정류장별_노선현황.csv')
df_08 = pd.read_csv(current_path+'/input/8.수원시_지하철역_위치정보.csv')
df_09 = pd.read_csv(current_path+'/input/9.수원시_지하철역별_이용현황(2017~2019).csv')
df_10 = pd.read_csv(current_path+'/input/10.수원시_옥외광고물현황.csv')
df_11 = pd.read_csv(current_path+'/input/11.수원시_대기오염도_측정현황.csv')
df_12 = pd.read_csv(current_path+'/input/12.수원시_주차장현황.csv')
df_13 = pd.read_csv(current_path+'/input/13.수원시_기상데이터(2020).csv')
df_14 = pd.read_csv(current_path+'/input/14.수원시_시간대별_유동인구(2020).csv')
df_15 = pd.read_csv(current_path+'/input/15.수원시_성연령별_유동인구(2020).csv')
df_16 = pd.read_csv(current_path+'/input/16.수원시_요일별_유동인구(2020).csv')
df_17 = gpd.read_file(current_path+'/input/17.수원시_인구정보(고령)_격자.geojson')
df_18 = gpd.read_file(current_path+'/input/18.수원시_인구정보(생산가능)_격자.geojson')
df_19 = gpd.read_file(current_path+'/input/19.수원시_인구정보(유소년)_격자.geojson')
df_20 = gpd.read_file(current_path+'/input/20.수원시_교통노드.geojson')
df_21 = gpd.read_file(current_path+'/input/21.수원시_교통링크.geojson')
df_22 = gpd.read_file(current_path+'/input/22.수원시_상세도로망_LV6.geojson')
df_23 = pd.read_csv(current_path+'/input/23.수원시_평일_일별_시간대별_추정교통량_LV6.csv')
df_24 = pd.read_csv(current_path+'/input/24.수원시_평일_일별_혼잡빈도강도_LV6.csv')
df_25 = pd.read_csv(current_path+'/input/25.수원시_평일_일별_혼잡시간강도_LV6.csv')
df_26 = gpd.read_file(current_path+'/input/26.수원시_인도(2017).geojson')
df_27 = gpd.read_file(current_path+'/input/27.수원시_도로명주소(건물).geojson')
df_28 = gpd.read_file(current_path+'/input/28.수원시_건물연면적_격자.geojson')
df_29 = gpd.read_file(current_path+'/input/29.수원시_법정경계(시군구).geojson')
df_30 = gpd.read_file(current_path+'/input/30.수원시_법정경계(읍면동).geojson')
df_31 = gpd.read_file(current_path+'/input/31.수원시_행정경계(읍면동).geojson')
df_32 = gpd.read_file(current_path+'/input/32.수원시_지적도.geojson')

In [3]:
# pydeck function 
def line_string_to_coordinates(line_string): 
    if isinstance(line_string, shapely.geometry.linestring.LineString): 
        lon, lat = line_string.xy 
        return [[x, y] for x, y in zip(lon, lat)] 
    elif isinstance(line_string, shapely.geometry.multilinestring.MultiLineString): 
        ret = [] 
        for i in range(len(line_string)): 
            lon, lat = line_string[i].xy 
            for x, y in zip(lon, lat): 
                ret.append([x, y])
        return ret 

def multipolygon_to_coordinates(x): 
    lon, lat = x[0].exterior.xy 
    return [[x, y] for x, y in zip(lon, lat)] 

def polygon_to_coordinates(x): 
    lon, lat = x.exterior.xy 
    return [[x, y] for x, y in zip(lon, lat)] 
 
def multipolygon_to_center_coordinates(x): 
    lon, lat = x[0].centroid.xy 
    return [[x, y] for x, y in zip(lon, lat)]

def polygon_to_center_coordinates(x): 
    lon, lat = x.centroid.xy
    return [[x, y] for x, y in zip(lon, lat)] 

In [4]:
token = "pk.eyJ1IjoiZGx3b3FsczQzMjMiLCJhIjoiY2tscnR3bG95MDJwaDJ2bjUzcTBrc3h4cyJ9.WigDFX0Gm612haaz4zQ2hg"

# 1. Bus Data  
* df_01 + df_07 
* df_02 ~ df_06 
---
* **경기 버스 정보** : http://www.gbis.go.kr/
* GGD_RouteInfo_M.xls : 버스 노선 정보 (기점, 종점 / 주중배차간격, 주말배차간격 / 첫차, 막차 시간) 
* GGD_RouteStationInfo_M.xls : 노선 경유 정보 (버스 노선 순서) 

## 1-1. df_01 + df_07 
* 버스 정보 합치기 
* 전처리 

In [5]:
# BIS = 1 & 인도 폭 0 or 4 이상인 정류장만 분석에 이용 
sidewalk = gpd.read_file(current_path+'/data/bus_filter_final.geojson')
sidewalk = sidewalk.drop(["layer", "path", "geometry"], axis=1)
sidewalk = sidewalk.drop_duplicates(["정류장ID"])

print(sidewalk.shape)
sidewalk.head()

(519, 14)


,정류장ID,정류장명,정류장유형,환승역타입,위치(동),쉘터,BIS설치여부,LED,LCD,LED+LCD복합형,알뜰형,임대형(음성),lon,lat
0,201000345,남수원초등학교,시내,일반,경기도 수원시 세류2동,nan,1,nan,nan,nan,1.000000,None,127.016000,37.251091
1,201000268,곡반중학교,시내,일반,경기도 수원시 곡선동,1.000000,1,1.000000,nan,nan,nan,None,127.034254,37.246081
2,201000357,수원은혜교회.한양수자인파크원아파트,시내,일반,경기도 수원시 금곡동,1.000000,1,nan,1.000000,nan,nan,None,126.953978,37.266016
3,202000090,화서역,"시내,마을",일반,경기도 수원시 화서2동,1.000000,1,nan,nan,1.000000,nan,None,126.990107,37.283895
4,202000092,월드메르디앙,시내,일반,경기도 수원시 우만2동,nan,1,nan,nan,1.000000,nan,None,127.037045,37.280164


In [6]:
tmp = sidewalk.merge(df_07[['정류소ID', '운행노선', '중앙차로여부']], how = 'left', left_on = '정류장ID', right_on = '정류소ID', copy = False)
del tmp['정류소ID']

In [7]:
tmp.head()

,정류장ID,정류장명,정류장유형,환승역타입,위치(동),쉘터,BIS설치여부,LED,LCD,LED+LCD복합형,알뜰형,임대형(음성),lon,lat,운행노선,중앙차로여부
0,201000345,남수원초등학교,시내,일반,경기도 수원시 세류2동,nan,1,nan,nan,nan,1.000000,None,127.016000,37.251091,82-1(수원여객),노변정류장
1,201000268,곡반중학교,시내,일반,경기도 수원시 곡선동,1.000000,1,1.000000,nan,nan,nan,None,127.034254,37.246081,"13-5(용남고속),15(제부여객),150(제부여객),151(제부여객),3002(용...",노변정류장
2,201000357,수원은혜교회.한양수자인파크원아파트,시내,일반,경기도 수원시 금곡동,1.000000,1,nan,1.000000,nan,nan,None,126.953978,37.266016,"11-1(수원여객),123A(용남고속),123D(용남고속),13-4(수원여객),15...",노변정류장
3,202000090,화서역,"시내,마을",일반,경기도 수원시 화서2동,1.000000,1,nan,nan,1.000000,nan,None,126.990107,37.283895,"16(수원여객),16-1(수원여객),16-2(수원여객),30(수원여객),30-1(수...",노변정류장
4,202000092,월드메르디앙,시내,일반,경기도 수원시 우만2동,nan,1,nan,nan,1.000000,nan,None,127.037045,37.280164,"3002(용남고속),7002((주)용남고속버스라인)",노변정류장


In [8]:
print(tmp.shape)
tmp.info()

(519, 16)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 519 entries, 0 to 518
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   정류장ID       519 non-null    int64  
 1   정류장명        519 non-null    object 
 2   정류장유형       519 non-null    object 
 3   환승역타입       513 non-null    object 
 4   위치(동)       519 non-null    object 
 5   쉘터          481 non-null    float64
 6   BIS설치여부     519 non-null    int64  
 7   LED         221 non-null    float64
 8   LCD         98 non-null     float64
 9   LED+LCD복합형  189 non-null    float64
 10  알뜰형         11 non-null     float64
 11  임대형(음성)     0 non-null      object 
 12  lon         519 non-null    float64
 13  lat         519 non-null    float64
 14  운행노선        518 non-null    object 
 15  중앙차로여부      518 non-null    object 
dtypes: float64(7), int64(2), object(7)
memory usage: 68.9+ KB


## 1-2. tmp data preprocessing 

In [9]:
# 결측값 확인 
tmp.isnull().sum()

정류장ID           0
정류장명            0
정류장유형           0
환승역타입           6
위치(동)           0
쉘터             38
BIS설치여부         0
LED           298
LCD           421
LED+LCD복합형    330
알뜰형           508
임대형(음성)       519
lon             0
lat             0
운행노선            1
중앙차로여부          1
dtype: int64

In [10]:
tmp.환승역타입 = '일반'
tmp.쉘터 = tmp.쉘터.fillna(0)

# LED / LCD / LED+LCD 
tmp.LED = tmp.LED.fillna(0)
tmp.LCD = tmp.LCD.fillna(0)
tmp['LED+LCD복합형'] = tmp['LED+LCD복합형'].fillna(0)

# 알뜰형 / 임대형(음성) 
tmp.알뜰형 = tmp.알뜰형.fillna(0)
tmp['임대형(음성)'] = tmp['임대형(음성)'].fillna(0)

In [11]:
# 버스가 안 다니는 정류장 ..
tmp[tmp.중앙차로여부.isna()]

,정류장ID,정류장명,정류장유형,환승역타입,위치(동),쉘터,BIS설치여부,LED,LCD,LED+LCD복합형,알뜰형,임대형(음성),lon,lat,운행노선,중앙차로여부
515,202000258,율천고등학교,"시내,마을",일반,경기도 수원시 화서2동,1.000000,1,1.000000,0.000000,0.000000,0.000000,0,126.978501,37.288431,NaN,NaN


In [12]:
# 운행노선 & 중앙차로여부 없는 데이터 drop 
tmp = tmp.dropna(axis=0)

In [13]:
print(tmp.shape)
tmp.head()

(518, 16)


,정류장ID,정류장명,정류장유형,환승역타입,위치(동),쉘터,BIS설치여부,LED,LCD,LED+LCD복합형,알뜰형,임대형(음성),lon,lat,운행노선,중앙차로여부
0,201000345,남수원초등학교,시내,일반,경기도 수원시 세류2동,0.000000,1,0.000000,0.000000,0.000000,1.000000,0,127.016000,37.251091,82-1(수원여객),노변정류장
1,201000268,곡반중학교,시내,일반,경기도 수원시 곡선동,1.000000,1,1.000000,0.000000,0.000000,0.000000,0,127.034254,37.246081,"13-5(용남고속),15(제부여객),150(제부여객),151(제부여객),3002(용...",노변정류장
2,201000357,수원은혜교회.한양수자인파크원아파트,시내,일반,경기도 수원시 금곡동,1.000000,1,0.000000,1.000000,0.000000,0.000000,0,126.953978,37.266016,"11-1(수원여객),123A(용남고속),123D(용남고속),13-4(수원여객),15...",노변정류장
3,202000090,화서역,"시내,마을",일반,경기도 수원시 화서2동,1.000000,1,0.000000,0.000000,1.000000,0.000000,0,126.990107,37.283895,"16(수원여객),16-1(수원여객),16-2(수원여객),30(수원여객),30-1(수...",노변정류장
4,202000092,월드메르디앙,시내,일반,경기도 수원시 우만2동,0.000000,1,0.000000,0.000000,1.000000,0.000000,0,127.037045,37.280164,"3002(용남고속),7002((주)용남고속버스라인)",노변정류장


In [14]:
# 결측값 확인 
tmp.isnull().sum()

정류장ID         0
정류장명          0
정류장유형         0
환승역타입         0
위치(동)         0
쉘터            0
BIS설치여부       0
LED           0
LCD           0
LED+LCD복합형    0
알뜰형           0
임대형(음성)       0
lon           0
lat           0
운행노선          0
중앙차로여부        0
dtype: int64

## 1-3. df_02 ~ df_06 

In [15]:
df_02_06 = pd.concat([df_02, df_03, df_04, df_05, df_06], axis=0, ignore_index = True)
df_02_06.shape

(2508607, 12)

In [16]:
# 노선번호 object 형태로 맞춰주기 
tqdm.pandas()
df_02_06["노선번호"] = df_02_06["노선번호"].progress_apply(lambda x : str(x))

100%|██████████| 2508607/2508607 [00:03<00:00, 754627.72it/s]


In [17]:
df_02_06.head()

,일자,업체명,노선번호,노선유형,정류소명,정류소ID,전체 승차 건수,초승 건수,환승 건수,전체 하차 건수,하차 건수,미태그 건수
0,20200101,(주)용남고속버스라인,3003,직행좌석형시내버스,서부공영차고지,201000446,1,1,0,0,0,0
1,20200101,(주)용남고속버스라인,3003,직행좌석형시내버스,구운동주민센터.서수원도서관,201000173,4,4,0,0,0,0
2,20200101,(주)용남고속버스라인,3003,직행좌석형시내버스,구운중학교,201000171,4,4,0,0,0,0
3,20200101,(주)용남고속버스라인,3003,직행좌석형시내버스,웃거리,201000031,5,4,1,0,0,0
4,20200101,(주)용남고속버스라인,3003,직행좌석형시내버스,구운오거리.경기중소기업성장지원센터,201000030,5,5,0,0,0,0


In [18]:
# '92' 처럼 용남고속 & 수원여객 동시운행 버스 체크 
df_02_06 = df_02_06.groupby(by=['일자', '정류소ID', '노선번호', '노선유형']).sum([['전체승차건수','초승건수','환승건수','전체하차건수','하차건수','미태그건수']]).reset_index()

In [19]:
print(df_02_06.shape)
df_02_06.head()

(2408422, 10)


,일자,정류소ID,노선번호,노선유형,전체 승차 건수,초승 건수,환승 건수,전체 하차 건수,하차 건수,미태그 건수
0,20200101,200000001,13,일반형시내버스,1,1,0,5,5,0
1,20200101,200000002,13,일반형시내버스,0,0,0,31,31,0
2,20200101,200000004,13,일반형시내버스,4,4,0,39,39,0
3,20200101,200000005,13,일반형시내버스,46,37,9,261,261,0
4,20200101,200000006,13,일반형시내버스,166,160,6,67,67,0


In [22]:
bus = df_02_06.merge(tmp, how='left', left_on = '정류소ID', right_on = '정류장ID', copy = True).drop(['정류장ID'], axis=1)

In [23]:
# BIS 미설치 정류장 & 인도 폭 기준 미달 정류장 제외 
bus = bus.dropna(axis=0)

In [24]:
print(bus.shape)
bus.head(2)

(1362748, 25)


,일자,정류소ID,노선번호,노선유형,전체 승차 건수,초승 건수,환승 건수,전체 하차 건수,하차 건수,미태그 건수,정류장명,정류장유형,환승역타입,위치(동),쉘터,BIS설치여부,LED,LCD,LED+LCD복합형,알뜰형,임대형(음성),lon,lat,운행노선,중앙차로여부
4,20200101,200000006,13,일반형시내버스,166,160,6,67,67,0,광교공원.경기대수원캠퍼스입구.연무시장,시내,일반,경기도 수원시 연무동,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,127.029464,37.300014,"13(수원여객),16(수원여객),16-1(수원여객),16-2(수원여객),32-5(수...",노변정류장
5,20200101,200000006,37,일반형시내버스,36,33,3,0,0,0,광교공원.경기대수원캠퍼스입구.연무시장,시내,일반,경기도 수원시 연무동,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,127.029464,37.300014,"13(수원여객),16(수원여객),16-1(수원여객),16-2(수원여객),32-5(수...",노변정류장


In [25]:
bus.isnull().sum()

일자            0
정류소ID         0
노선번호          0
노선유형          0
전체 승차 건수      0
초승 건수         0
환승 건수         0
전체 하차 건수      0
하차 건수         0
미태그 건수        0
정류장명          0
정류장유형         0
환승역타입         0
위치(동)         0
쉘터            0
BIS설치여부       0
LED           0
LCD           0
LED+LCD복합형    0
알뜰형           0
임대형(음성)       0
lon           0
lat           0
운행노선          0
중앙차로여부        0
dtype: int64

In [26]:
print("df_01 unique 정류장 ID 개수 : ", df_01["정류장ID"].nunique())
print("df_02~06 unique 정류장 ID 개수 : ", df_02_06["정류소ID"].nunique())

print("=====" * 20)
print("합치기 전 unique 정류장 ID 개수 : ", tmp["정류장ID"].nunique())
print("합친 후의 unique 정류장 ID 개수 : ", bus["정류소ID"].nunique())

df_01 unique 정류장 ID 개수 :  1179
df_02~06 unique 정류장 ID 개수 :  1114
합치기 전 unique 정류장 ID 개수 :  518
합친 후의 unique 정류장 ID 개수 :  516


## 1-4. 외부데이터 

In [94]:
route = pd.read_excel(current_path+'/data/GGD_RouteInfo_M.xls')
trans = pd.read_excel(current_path+'/data/GGD_RouteStationInfo_M.xls', sheet_name=[0,1,2])

In [95]:
trans = pd.concat([trans[0], trans[1], trans[2]], axis=0, ignore_index = True)

### (1) 버스 노선 순서 데이터 

In [96]:
# 버스 노선 순서 데이터 
trans2 = trans.rename(columns = {"STATION_ID": "정류소ID", "ROUTE_ID": "노선ID", "ROUTE_NM": "노선번호", "STA_ORDER": "노선순서", "STATION_NM": "정류장명"})
trans2 = trans2.drop(["순번", "MOBILE_NO"], axis=1)
trans2.head(3)

,노선ID,노선번호,노선순서,정류장명,정류소ID
0,200000006,300,1,하북차고지,214001151
1,200000006,300,2,서탄입구,214000514
2,200000006,300,3,진위역,214000513


In [103]:
trans2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192708 entries, 0 to 192707
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   노선ID    192708 non-null  int64 
 1   노선번호    192708 non-null  object
 2   노선순서    192708 non-null  int64 
 3   정류장명    192708 non-null  object
 4   정류소ID   192708 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 7.4+ MB


In [105]:
trans2["노선번호"] = trans2["노선번호"].progress_apply(lambda x : str(x))
trans2["노선번호"] = trans2["노선번호"].progress_apply(lambda x : x.replace("-", "_"))

100%|██████████| 192708/192708 [00:00<00:00, 618964.90it/s]


In [106]:
# Data Merge 
tmp2 = bus.merge(trans2.drop(["정류장명"], axis=1), how='left', on=['정류소ID', '노선번호'])
print("bus+trans shape : ", tmp2.shape , '\n') 
print("원래 데이터의 unique bus 개수 : ", bus.정류소ID.nunique())
print("합친 데이터의 unique bus 개수 : ", tmp2.정류소ID.nunique())
print("=====" * 20)
tmp2.head()

bus+trans shape :  (1364613, 27) 

원래 데이터의 unique bus 개수 :  516
합친 데이터의 unique bus 개수 :  516


,일자,정류소ID,노선번호,노선유형,전체 승차 건수,초승 건수,환승 건수,전체 하차 건수,하차 건수,미태그 건수,정류장명,정류장유형,환승역타입,위치(동),쉘터,BIS설치여부,LED,LCD,LED+LCD복합형,알뜰형,임대형(음성),lon,lat,운행노선,중앙차로여부,노선ID,노선순서
0,20200101,200000006,13,일반형시내버스,166,160,6,67,67,0,광교공원.경기대수원캠퍼스입구.연무시장,시내,일반,경기도 수원시 연무동,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,127.029464,37.300014,"13(수원여객),16(수원여객),16-1(수원여객),16-2(수원여객),32-5(수...",노변정류장,200000037.000000,13.000000
1,20200101,200000006,37,일반형시내버스,36,33,3,0,0,0,광교공원.경기대수원캠퍼스입구.연무시장,시내,일반,경기도 수원시 연무동,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,127.029464,37.300014,"13(수원여객),16(수원여객),16-1(수원여객),16-2(수원여객),32-5(수...",노변정류장,200000099.000000,1.000000
2,20200101,200000006,7_1,일반형시내버스,187,162,25,190,190,0,광교공원.경기대수원캠퍼스입구.연무시장,시내,일반,경기도 수원시 연무동,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,127.029464,37.300014,"13(수원여객),16(수원여객),16-1(수원여객),16-2(수원여객),32-5(수...",노변정류장,200000045.000000,55.000000
3,20200101,200000008,13,일반형시내버스,127,127,0,46,46,0,문암골,시내,일반,경기도 수원시 연무동,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,127.027700,37.308946,13(수원여객),노변정류장,200000037.000000,11.000000
4,20200101,200000036,112,일반형시내버스,36,34,2,76,76,0,풍림아파트입구,"시내,마을",일반,경기도 수원시 정자3동,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,126.995257,37.297929,112(수원여객),노변정류장,200000049.000000,29.000000


### (2) 버스 노선 정보 데이터 

In [107]:
route.head(3)

,순번,관할관청,운행업체,노선번호,노선ID,기점,기점_STATION_ID,종점,종점_STATION_ID,주중배차간격,주말배차간격,상행첫차,상행막차,하행첫차,하행막차
0,1,가평군,가평교통,50-5,239000002,목동터미널,239000272,용수동종점,239000052,370분,370분,09:20,19:00,10:10,19:50
1,2,가평군,가평교통,40-5,239000005,현리터미널,239000687,귀목종점,239000123,270분,270분,06:40,17:00,07:10,17:30
2,3,가평군,가평교통,43,239000006,현리터미널,239000687,청평터미널,239000738,120분,120분,06:00,19:00,06:20,19:30


In [108]:
route.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3480 entries, 0 to 3479
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   순번             3480 non-null   int64 
 1   관할관청           3480 non-null   object
 2   운행업체           3478 non-null   object
 3   노선번호           3480 non-null   object
 4   노선ID           3480 non-null   int64 
 5   기점             3480 non-null   object
 6   기점_STATION_ID  3480 non-null   int64 
 7   종점             3480 non-null   object
 8   종점_STATION_ID  3480 non-null   int64 
 9   주중배차간격         2596 non-null   object
 10  주말배차간격         2334 non-null   object
 11  상행첫차           3340 non-null   object
 12  상행막차           3328 non-null   object
 13  하행첫차           3139 non-null   object
 14  하행막차           3144 non-null   object
dtypes: int64(4), object(11)
memory usage: 407.9+ KB


In [109]:
route["노선번호"] = route["노선번호"].progress_apply(lambda x : str(x))
route["노선번호"] = route["노선번호"].progress_apply(lambda x : x.replace("-", "_"))

100%|██████████| 3480/3480 [00:00<00:00, 614343.11it/s]


In [110]:
route.shape

(3480, 15)

In [111]:
route = route[route["하행첫차"] != '1899/1']
print(route.shape)

(3479, 15)


In [112]:
# 상행첫차, 상행막차, 하행첫차, 하행막차 type 바꿔주기 
route['상행첫차'] = route['상행첫차'].progress_apply(lambda x: pd.to_datetime(str(x), format='%H:%M'))
route['상행막차'] = route['상행막차'].progress_apply(lambda x: pd.to_datetime(str(x), format='%H:%M'))
route['하행첫차'] = route['하행첫차'].progress_apply(lambda x: pd.to_datetime(str(x), format='%H:%M'))
route['하행막차'] = route['하행막차'].progress_apply(lambda x: pd.to_datetime(str(x), format='%H:%M'))

100%|██████████| 3479/3479 [00:00<00:00, 5542.31it/s]


In [113]:
# 운행시간 구하기 
def operation_time(first, last) : 
    if last >= first  : 
        return last - first 
    else : 
        last += timedelta(days=1)
        return last - first 

In [114]:
route["상행운행시간"] = route.progress_apply(lambda x : operation_time(x["상행첫차"], x["상행막차"]), axis=1)
route["하행운행시간"] = route.progress_apply(lambda x : operation_time(x["하행첫차"], x["하행막차"]), axis=1)

100%|██████████| 3479/3479 [00:00<00:00, 13454.95it/s]


In [116]:
# Data Merge 
tmp3 = pd.merge(left=tmp2, right=route.drop(["순번"], axis=1).drop_duplicates('노선ID'), how='left', \
                on=['노선ID','노선번호'])

print("bus+trans+route shape : ", tmp3.shape , '\n') 
print("bus 데이터의 unique bus 개수 : ", bus.정류소ID.nunique())
print("tmp2 데이터의 unique bus 개수 : ", tmp2.정류소ID.nunique())
print("합친 데이터의 unique bus 개수 : ", tmp3.정류소ID.nunique())
print("=====" * 20)
tmp3.head()

bus+trans+route shape :  (1364613, 41) 

bus 데이터의 unique bus 개수 :  516
tmp2 데이터의 unique bus 개수 :  516
합친 데이터의 unique bus 개수 :  516


,일자,정류소ID,노선번호,노선유형,전체 승차 건수,초승 건수,환승 건수,전체 하차 건수,하차 건수,미태그 건수,정류장명,정류장유형,환승역타입,위치(동),쉘터,BIS설치여부,LED,LCD,LED+LCD복합형,알뜰형,임대형(음성),lon,lat,운행노선,중앙차로여부,노선ID,노선순서,관할관청,운행업체,기점,기점_STATION_ID,종점,종점_STATION_ID,주중배차간격,주말배차간격,상행첫차,상행막차,하행첫차,하행막차,상행운행시간,하행운행시간
0,20200101,200000006,13,일반형시내버스,166,160,6,67,67,0,광교공원.경기대수원캠퍼스입구.연무시장,시내,일반,경기도 수원시 연무동,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,127.029464,37.300014,"13(수원여객),16(수원여객),16-1(수원여객),16-2(수원여객),32-5(수...",노변정류장,200000037.000000,13.000000,수원시,수원여객,상광교동종점,200000275.000000,칠보고등학교.칠보중학교.극동스타클래스,201000099.000000,5분~7분,5분~7분,1900-01-01 06:00:00,1900-01-01 22:00:00,1900-01-01 05:40:00,1900-01-01 23:10:00,0 days 16:00:00,0 days 17:30:00
1,20200101,200000006,37,일반형시내버스,36,33,3,0,0,0,광교공원.경기대수원캠퍼스입구.연무시장,시내,일반,경기도 수원시 연무동,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,127.029464,37.300014,"13(수원여객),16(수원여객),16-1(수원여객),16-2(수원여객),32-5(수...",노변정류장,200000099.000000,1.000000,수원시,수원여객,광교공원.경기대수원캠퍼스입구.연무시장,200000006.000000,한국민속촌.보라해링턴,228001646.000000,10분~13분,15분~20분,1900-01-01 04:50:00,1900-01-01 21:40:00,1900-01-01 06:00:00,1900-01-01 23:10:00,0 days 16:50:00,0 days 17:10:00
2,20200101,200000006,7_1,일반형시내버스,187,162,25,190,190,0,광교공원.경기대수원캠퍼스입구.연무시장,시내,일반,경기도 수원시 연무동,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,127.029464,37.300014,"13(수원여객),16(수원여객),16-1(수원여객),16-2(수원여객),32-5(수...",노변정류장,200000045.000000,55.000000,수원시,수원여객,동탄차고지,233001222.000000,광교공원.경기대수원캠퍼스입구.연무시장,200000006.000000,7분~9분,9분~12분,1900-01-01 05:00:00,1900-01-01 22:40:00,1900-01-01 06:00:00,1900-01-01 23:45:00,0 days 17:40:00,0 days 17:45:00
3,20200101,200000008,13,일반형시내버스,127,127,0,46,46,0,문암골,시내,일반,경기도 수원시 연무동,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,127.027700,37.308946,13(수원여객),노변정류장,200000037.000000,11.000000,수원시,수원여객,상광교동종점,200000275.000000,칠보고등학교.칠보중학교.극동스타클래스,201000099.000000,5분~7분,5분~7분,1900-01-01 06:00:00,1900-01-01 22:00:00,1900-01-01 05:40:00,1900-01-01 23:10:00,0 days 16:00:00,0 days 17:30:00
4,20200101,200000036,112,일반형시내버스,36,34,2,76,76,0,풍림아파트입구,"시내,마을",일반,경기도 수원시 정자3동,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,126.995257,37.297929,112(수원여객),노변정류장,200000049.000000,29.000000,수원시,수원여객,곡반정동차고지,201000143.000000,웅비아파트,201000330.000000,7분~9분,9분~12분,1900-01-01 05:00:00,1900-01-01 22:50:00,1900-01-01 06:05:00,1900-01-01 23:55:00,0 days 17:50:00,0 days 17:50:00


## 1-5. Reduce Memory Usage 

In [58]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df                

In [117]:
tmp3 = reduce_mem_usage(tmp3)

Mem. usage decreased to 285.01 Mb (34.8% reduction)


In [118]:
tmp3.isnull().sum()

일자                   0
정류소ID                0
노선번호                 0
노선유형                 0
전체 승차 건수             0
초승 건수                0
환승 건수                0
전체 하차 건수             0
하차 건수                0
미태그 건수               0
정류장명                 0
정류장유형                0
환승역타입                0
위치(동)                0
쉘터                   0
BIS설치여부              0
LED                  0
LCD                  0
LED+LCD복합형           0
알뜰형                  0
임대형(음성)              0
lon                  0
lat                  0
운행노선                 0
중앙차로여부               0
노선ID             33900
노선순서             33900
관할관청             33900
운행업체             33900
기점               33900
기점_STATION_ID    33900
종점               33900
종점_STATION_ID    33900
주중배차간격           36524
주말배차간격           38510
상행첫차             33900
상행막차             33900
하행첫차             35748
하행막차             35748
상행운행시간           33900
하행운행시간           35748
dtype: int64

## 1-6. dataframe split 
* 결측값 문제를 해결하기 위해 일단 데이터 split 

In [148]:
# 결측값 없는 dataframe index 
idx = tmp3.dropna(axis=0).index

final = tmp3.loc[idx]
final_na = tmp3[~tmp3.index.isin(idx)]

# check 
print(final.shape, final_na.shape)
tmp3.shape[0] == (final.shape[0] + final_na.shape[0])

(1326103, 41) (38510, 41)


True

In [149]:
final.isnull().sum()

일자               0
정류소ID            0
노선번호             0
노선유형             0
전체 승차 건수         0
초승 건수            0
환승 건수            0
전체 하차 건수         0
하차 건수            0
미태그 건수           0
정류장명             0
정류장유형            0
환승역타입            0
위치(동)            0
쉘터               0
BIS설치여부          0
LED              0
LCD              0
LED+LCD복합형       0
알뜰형              0
임대형(음성)          0
lon              0
lat              0
운행노선             0
중앙차로여부           0
노선ID             0
노선순서             0
관할관청             0
운행업체             0
기점               0
기점_STATION_ID    0
종점               0
종점_STATION_ID    0
주중배차간격           0
주말배차간격           0
상행첫차             0
상행막차             0
하행첫차             0
하행막차             0
상행운행시간           0
하행운행시간           0
dtype: int64

## 1-6. 추가 전처리 

In [150]:
# 일자 data type 바꿔주기 (오래걸림)
final['일자'] = final['일자'].progress_apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))

100%|██████████| 1326103/1326103 [04:28<00:00, 4932.86it/s]


In [151]:
# 중앙차로여부 
final["중앙차로여부"] = final["중앙차로여부"].progress_apply(lambda x : 1 if x == '중앙차로 정류장' else 0)

100%|██████████| 1326103/1326103 [00:01<00:00, 709671.91it/s]


In [152]:
# 운행노선 / 정류장유형 
final["운행노선"] = final["운행노선"].progress_apply(lambda x : x.split(','))
final["운행노선수"] = final["운행노선"].progress_apply(lambda x : len(x))

final["정류장유형"] = final["정류장유형"].progress_apply(lambda x : x.split(','))
final["정류장유형수"] = final["정류장유형"].progress_apply(lambda x : len(x))

100%|██████████| 1326103/1326103 [00:01<00:00, 716204.30it/s]


## 1-7. Data Save 

In [65]:
Q1 = (tmp3['일자'] >= '2020-01-01') & (tmp3['일자'] <= '2020-03-31')
Q2 = (tmp3['일자'] >= '2020-04-01') & (tmp3['일자'] <= '2020-06-30')
Q3 = (tmp3['일자'] >= '2020-07-01') & (tmp3['일자'] <= '2020-09-30')
Q4 = (tmp3['일자'] >= '2020-10-01') & (tmp3['일자'] <= '2020-12-31')

In [66]:
Q1_df, Q2_df, Q3_df, Q4_df = tmp3.loc[Q1], tmp3.loc[Q2], tmp3.loc[Q3], tmp3.loc[Q4]
Q1_df.shape, Q2_df.shape, Q3_df.shape, Q4_df.shape

((325434, 43), (340074, 43), (346872, 43), (351871, 43))

In [67]:
# check 
tmp3.shape[0] == (Q1_df.shape[0] + Q2_df.shape[0] + Q3_df.shape[0] + Q4_df.shape[0])

True

In [68]:
# 저장 
Q1_df.to_excel(current_path+'/busdata/bus_Q1.xlsx', index=False)
Q2_df.to_excel(current_path+'/busdata/bus_Q2.xlsx', index=False)
Q3_df.to_excel(current_path+'/busdata/bus_Q3.xlsx', index=False)
Q4_df.to_excel(current_path+'/busdata/bus_Q4.xlsx', index=False)

In [ ]:
# 환승많이 / 승차많이 : 승차에 비해 환승이 n배 이상 많은 정류장 등 ... 

# 2. Bus Data Feature Engineering 
* tmp : 버스 정류장 고유 정보 데이터 (518, 16)
* total : 일별 노선 승하차 정보 포함 데이터 (1364251, 43)
--- 
* **tmp** 데이터에 맞춰서 (**버스 정류장ID** 기준으로), 데이터 만들기 
* 첫차시간: 가장 빠른 첫차시간 / 막차시간 : 가장 느린 막차시간 
---
* 주중 / 주말 운행 버스 개수 

In [69]:
Q1_df = pd.read_excel(current_path+'/busdata/bus_Q1.xlsx')
Q2_df = pd.read_excel(current_path+'/busdata/bus_Q2.xlsx')
Q3_df = pd.read_excel(current_path+'/busdata/bus_Q3.xlsx')
Q4_df = pd.read_excel(current_path+'/busdata/bus_Q4.xlsx')

Q1_df.shape, Q2_df.shape, Q3_df.shape, Q4_df.shape

((325434, 43), (340074, 43), (346872, 43), (351871, 43))

In [70]:
total = pd.concat([Q1_df, Q2_df, Q3_df, Q4_df], axis=0)
total.shape 

(1364251, 43)

In [71]:
# check 
total.shape[0] == (Q1_df.shape[0] + Q2_df.shape[0] + Q3_df.shape[0] + Q4_df.shape[0])

True

In [75]:
print(tmp.shape)
tmp.head()

(518, 16)


,정류장ID,정류장명,정류장유형,환승역타입,위치(동),쉘터,BIS설치여부,LED,LCD,LED+LCD복합형,알뜰형,임대형(음성),lon,lat,운행노선,중앙차로여부
0,201000345,남수원초등학교,시내,일반,경기도 수원시 세류2동,0.000000,1,0.000000,0.000000,0.000000,1.000000,0,127.016000,37.251091,82-1(수원여객),노변정류장
1,201000268,곡반중학교,시내,일반,경기도 수원시 곡선동,1.000000,1,1.000000,0.000000,0.000000,0.000000,0,127.034254,37.246081,"13-5(용남고속),15(제부여객),150(제부여객),151(제부여객),3002(용...",노변정류장
2,201000357,수원은혜교회.한양수자인파크원아파트,시내,일반,경기도 수원시 금곡동,1.000000,1,0.000000,1.000000,0.000000,0.000000,0,126.953978,37.266016,"11-1(수원여객),123A(용남고속),123D(용남고속),13-4(수원여객),15...",노변정류장
3,202000090,화서역,"시내,마을",일반,경기도 수원시 화서2동,1.000000,1,0.000000,0.000000,1.000000,0.000000,0,126.990107,37.283895,"16(수원여객),16-1(수원여객),16-2(수원여객),30(수원여객),30-1(수...",노변정류장
4,202000092,월드메르디앙,시내,일반,경기도 수원시 우만2동,0.000000,1,0.000000,0.000000,1.000000,0.000000,0,127.037045,37.280164,"3002(용남고속),7002((주)용남고속버스라인)",노변정류장


### (1) 배차 간격 

In [153]:
# 간격은 다 '분' 으로 표현 
final[(final["주중배차간격"].str.contains("~")) & (final["주중배차간격"].str.contains("분") == False)].shape

(0, 43)

In [155]:
# 배차간격 
def interval(x) : 
    try : 
        itv = x.split("~")
        a, b = int(itv[0][:-1]), int(itv[1][:-1])
        return (a+b)/2
    except :
        return int(x[:-1])

In [156]:
# 분 단위 배차간격 
final["주중배차간격(분)"] = final["주중배차간격"].progress_apply(lambda x : interval(x))
final["주말배차간격(분)"] = final["주말배차간격"].progress_apply(lambda x : interval(x))

100%|██████████| 1326103/1326103 [00:03<00:00, 332825.91it/s]


### (2) 배차 횟수

In [157]:
# 배차횟수 = 운행시간 / 배차간격 
def operation_num(duration, interval) : 
    duration2, interval2 = duration.total_seconds() , interval * 60
    return duration2 / interval2

In [159]:
final["주중상행배차횟수"] = final.progress_apply(lambda x : operation_num(x["상행운행시간"], x["주중배차간격(분)"]), axis=1)
final["주중하행배차횟수"] = final.progress_apply(lambda x : operation_num(x["하행운행시간"], x["주중배차간격(분)"]), axis=1)

final["주말상행배차횟수"] = final.progress_apply(lambda x : operation_num(x["상행운행시간"], x["주말배차간격(분)"]), axis=1)
final["주말하행배차횟수"] = final.progress_apply(lambda x : operation_num(x["하행운행시간"], x["주말배차간격(분)"]), axis=1)

100%|██████████| 1326103/1326103 [01:34<00:00, 13999.55it/s]


In [163]:
final.shape

(1326103, 49)

In [166]:
print(tmp.shape)
tmp.head()

(518, 16)


,정류장ID,정류장명,정류장유형,환승역타입,위치(동),쉘터,BIS설치여부,LED,LCD,LED+LCD복합형,알뜰형,임대형(음성),lon,lat,운행노선,중앙차로여부
0,201000345,남수원초등학교,시내,일반,경기도 수원시 세류2동,0.000000,1,0.000000,0.000000,0.000000,1.000000,0,127.016000,37.251091,82-1(수원여객),노변정류장
1,201000268,곡반중학교,시내,일반,경기도 수원시 곡선동,1.000000,1,1.000000,0.000000,0.000000,0.000000,0,127.034254,37.246081,"13-5(용남고속),15(제부여객),150(제부여객),151(제부여객),3002(용...",노변정류장
2,201000357,수원은혜교회.한양수자인파크원아파트,시내,일반,경기도 수원시 금곡동,1.000000,1,0.000000,1.000000,0.000000,0.000000,0,126.953978,37.266016,"11-1(수원여객),123A(용남고속),123D(용남고속),13-4(수원여객),15...",노변정류장
3,202000090,화서역,"시내,마을",일반,경기도 수원시 화서2동,1.000000,1,0.000000,0.000000,1.000000,0.000000,0,126.990107,37.283895,"16(수원여객),16-1(수원여객),16-2(수원여객),30(수원여객),30-1(수...",노변정류장
4,202000092,월드메르디앙,시내,일반,경기도 수원시 우만2동,0.000000,1,0.000000,0.000000,1.000000,0.000000,0,127.037045,37.280164,"3002(용남고속),7002((주)용남고속버스라인)",노변정류장


### (3) 통계량 

In [179]:
final2 = final.drop(['정류장명', '정류장유형', '환승역타입', '위치(동)', '쉘터', 'BIS설치여부', 
                     'LED', 'LCD', 'LED+LCD복합형', '알뜰형', '임대형(음성)', 'lon', 'lat', 
                     '노선ID', '기점_STATION_ID', '종점_STATION_ID'], axis=1)

In [238]:
temp = final2.groupby('정류소ID').agg({
    '전체 승차 건수' :  [('전체승차건수_max', np.max),
                    ('전체승차건수_min', np.min),
                    ('전체승차건수_mean', np.mean),
                    ('전체승차건수_median', np.median)
                    ], 
    
    '초승 건수' : [('초승건수_max', np.max),
                ('초승건수_min', np.min),
                ('초승건수_mean', np.mean),
                ('초승건수_median', np.median)
                ],  
    
    '환승 건수' : [('환승건수_max', np.max),
                ('환승건수_min', np.min),
                ('환승건수_mean', np.mean),
                ('환승건수_median', np.median)
                ],      
    
    '전체 하차 건수' : [('전체하차건수_max', np.max),
                ('전체하차건수_min', np.min),
                ('전체하차건수_mean', np.mean),
                ('전체하차건수_median', np.median)
                ],  
    
    '하차 건수' : [('하차건수_max', np.max),
                ('하차건수_min', np.min),
                ('하차건수_mean', np.mean),
                ('하차건수_median', np.median)
                ], 
    
    '미태그 건수' : [('미태그건수_max', np.max),
                ('미태그건수_min', np.min),
                ('미태그건수_mean', np.mean),
                ('미태그건수_median', np.median)
                ],     

    '노선순서' : [('노선순서_max', np.max),
                ('노선순서_min', np.min),
                ('노선순서_mean', np.mean),
                ('노선순서_median', np.median)
                ],     

    '주중배차간격(분)' : [('주중배차간격(분)_max', np.max),
                ('주중배차간격(분)_min', np.min),
                ('주중배차간격(분)_mean', np.mean),
                ('주중배차간격(분)_median', np.median)
                ],         

    '주말배차간격(분)' : [('주말배차간격(분)_max', np.max),
                ('주말배차간격(분)_min', np.min),
                ('주말배차간격(분)_mean', np.mean),
                ('주말배차간격(분)_median', np.median)
                ],         

    '주중상행배차횟수' : [('주중상행배차횟수_max', np.max),
                ('주중상행배차횟수_min', np.min),
                ('주중상행배차횟수_mean', np.mean),
                ('주중상행배차횟수_median', np.median)
                ],         

    '주중하행배차횟수' : [('주중하행배차횟수_max', np.max),
                ('주중하행배차횟수_min', np.min),
                ('주중하행배차횟수_mean', np.mean),
                ('주중하행배차횟수_median', np.median)
                ],         


    '주말상행배차횟수' : [('주말상행배차횟수_max', np.max),
                ('주말상행배차횟수_min', np.min),
                ('주말상행배차횟수_mean', np.mean),
                ('주말상행배차횟수_median', np.median)
                ],             
    
    '주말하행배차횟수' : [('주말하행배차횟수_max', np.max),
                ('주말하행배차횟수_min', np.min),
                ('주말하행배차횟수_mean', np.mean),
                ('주말하행배차횟수_median', np.median)
                ]
    
}).reset_index()

In [240]:
print(temp.shape)
temp.head()

(516, 53)


정류소ID   전체 승차 건수                                         초승 건수                                   환승 건수                                  전체 하차 건수                                         하차 건수                                   미태그 건수                                         노선순서                                     주중배차간격(분)                                                   주말배차간격(분)                                                   주중상행배차횟수                                                주중하행배차횟수                                                주말상행배차횟수                                                주말하행배차횟수                                           
             전체승차건수_max 전체승차건수_min 전체승차건수_mean 전체승차건수_median 초승건수_max 초승건수_min 초승건수_mean 초승건수_median 환승건수_max 환승건수_min 환승건수_mean 환승건수_median 전체하차건수_max 전체하차건수_min 전체하차건수_mean 전체하차건수_median 하차건수_max 하차건수_min 하차건수_mean 하차건수_median 미태그건수_max 미태그건수_min 미태그건수_mean 미태그건수_median   노선순서_max  노선순서_min 노선순서_mean 노선순서_median 주중배차간격(분)_max 주중배차간격(분)_min 주중배차간격(분)_mean 주중배차간격(분)_median 주말배차간격(분)_max 주말배차간격(분)_min 주말배차간격(분)_mean 주말배차간격(분)_median 주중상행배차횟수_max 주중상행배차횟수_min 주중상행배차횟수_mean 주중상행배차횟수_median 주중하행배차횟수_max 주중하행배차횟수_min 주중하행배차횟수_mean 주중하행배차횟수_median 주말상행배차횟수_max 주말상행배차횟수_min 주말상행배차횟수_mean 주말상행배차횟수_median 주말하행배차횟수_max 주말하행배차횟수_min 주말하행배차횟수_mean 주말하행배차횟수_median
0  200000006        497          0   91.183938     48.000000      471        0 84.721934   44.000000      116        0  6.462003    3.000000        432          0   42.409758      1.000000      432        0 42.409758    1.000000         0         0          0            0  55.000000  1.000000 11.929688    2.000000    210.000000      6.000000      79.955743        11.500000    210.000000      6.000000      81.545769        17.500000   160.000000     3.833333     73.886349       87.826087   175.000000     3.833333     76.561834       89.565217   160.000000     3.833333     62.330032       57.714286   175.000000     3.833333     64.896849       58.857143
1  200000008        206          8   77.498630     69.000000      206        8 77.200000   69.000000        6        0  0.298630    0.000000        164          0   36.594521     21.000000      164        0 36.594521   21.000000         0         0          0            0  11.000000 11.000000 11.000000   11.000000      6.000000      6.000000       6.000000         6.000000      6.000000      6.000000       6.000000         6.000000   160.000000   160.000000    160.000000      160.000000   175.000000   175.000000    175.000000      175.000000   160.000000   160.000000    160.000000      160.000000   175.000000   175.000000    175.000000      175.000000
2  200000021        146         14   77.862259     80.000000      136       11 72.944904   74.000000       16        0  4.917355    5.000000        127         10   65.815427     66.000000      127       10 65.815427   66.000000         0         0          0            0  67.000000 67.000000 67.000000   67.000000      8.000000      8.000000       8.000000         8.000000     10.500000     10.500000      10.500000        10.500000   133.750000   133.750000    133.750000      133.750000   133.750000   133.750000    133.750000      133.750000   101.904762   101.904762    101.904762      101.904762   101.904762   101.904762    101.904762      101.904762
3  200000036        133         15   73.186301     74.000000      119       12 66.775342   68.000000       17        0  6.410959    6.000000        174         28   99.109589    102.000000      174       28 99.109589  102.000000         0         0          0            0  29.000000 29.000000 29.000000   29.000000      8.000000      8.000000       8.000000         8.000000     10.500000     10.500000      10.500000        10.500000   133.750000   133.750000    133.750000      133.750000   133.750000   133.750000    133.750000      133.750000   101.904762   101.904762    101.904762      101.904762   101.904762   101.904762    101.904762      101.904762
4  200000047        152          0   24.285463 

In [252]:
temp.columns = temp.columns.get_level_values(level=1)

In [253]:
print(temp.shape)
temp.head()

(516, 53)


,,전체승차건수_max,전체승차건수_min,전체승차건수_mean,전체승차건수_median,초승건수_max,초승건수_min,초승건수_mean,초승건수_median,환승건수_max,환승건수_min,환승건수_mean,환승건수_median,전체하차건수_max,전체하차건수_min,전체하차건수_mean,전체하차건수_median,하차건수_max,하차건수_min,하차건수_mean,하차건수_median,미태그건수_max,미태그건수_min,미태그건수_mean,미태그건수_median,노선순서_max,노선순서_min,노선순서_mean,노선순서_median,주중배차간격(분)_max,주중배차간격(분)_min,주중배차간격(분)_mean,주중배차간격(분)_median,주말배차간격(분)_max,주말배차간격(분)_min,주말배차간격(분)_mean,주말배차간격(분)_median,주중상행배차횟수_max,주중상행배차횟수_min,주중상행배차횟수_mean,주중상행배차횟수_median,주중하행배차횟수_max,주중하행배차횟수_min,주중하행배차횟수_mean,주중하행배차횟수_median,주말상행배차횟수_max,주말상행배차횟수_min,주말상행배차횟수_mean,주말상행배차횟수_median,주말하행배차횟수_max,주말하행배차횟수_min,주말하행배차횟수_mean,주말하행배차횟수_median
0,200000006,497,0,91.183938,48.000000,471,0,84.721934,44.000000,116,0,6.462003,3.000000,432,0,42.409758,1.000000,432,0,42.409758,1.000000,0,0,0,0,55.000000,1.000000,11.929688,2.000000,210.000000,6.000000,79.955743,11.500000,210.000000,6.000000,81.545769,17.500000,160.000000,3.833333,73.886349,87.826087,175.000000,3.833333,76.561834,89.565217,160.000000,3.833333,62.330032,57.714286,175.000000,3.833333,64.896849,58.857143
1,200000008,206,8,77.498630,69.000000,206,8,77.200000,69.000000,6,0,0.298630,0.000000,164,0,36.594521,21.000000,164,0,36.594521,21.000000,0,0,0,0,11.000000,11.000000,11.000000,11.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,160.000000,160.000000,160.000000,160.000000,175.000000,175.000000,175.000000,175.000000,160.000000,160.000000,160.000000,160.000000,175.000000,175.000000,175.000000,175.000000
2,200000021,146,14,77.862259,80.000000,136,11,72.944904,74.000000,16,0,4.917355,5.000000,127,10,65.815427,66.000000,127,10,65.815427,66.000000,0,0,0,0,67.000000,67.000000,67.000000,67.000000,8.000000,8.000000,8.000000,8.000000,10.500000,10.500000,10.500000,10.500000,133.750000,133.750000,133.750000,133.750000,133.750000,133.750000,133.750000,133.750000,101.904762,101.904762,101.904762,101.904762,101.904762,101.904762,101.904762,101.904762
3,200000036,133,15,73.186301,74.000000,119,12,66.775342,68.000000,17,0,6.410959,6.000000,174,28,99.109589,102.000000,174,28,99.109589,102.000000,0,0,0,0,29.000000,29.000000,29.000000,29.000000,8.000000,8.000000,8.000000,8.000000,10.500000,10.500000,10.500000,10.500000,133.750000,133.750000,133.750000,133.750000,133.750000,133.750000,133.750000,133.750000,101.904762,101.904762,101.904762,101.904762,101.904762,101.904762,101.904762,101.904762
4,200000047,152,0,24.285463,1.000000,146,0,23.975381,1.000000,8,0,0.310082,0.000000,165,0,24.930832,5.000000,165,0,24.930832,5.000000,0,0,0,0,117.000000,41.000000,72.562500,56.000000,210.000000,13.000000,116.211899,135.000000,210.000000,24.000000,119.100234,135.000000,82.307692,3.833333,27.346116,7.481481,83.846154,3.833333,27.997867,7.481481,44.583333,3.833333,18.781229,7.481481,45.416667,3.833333,19.258605,7.481481


In [255]:
temp = temp.rename(columns = {'' : '정류소ID'})

In [256]:
print(temp.shape)
temp.head()

(516, 53)


,정류소ID,전체승차건수_max,전체승차건수_min,전체승차건수_mean,전체승차건수_median,초승건수_max,초승건수_min,초승건수_mean,초승건수_median,환승건수_max,환승건수_min,환승건수_mean,환승건수_median,전체하차건수_max,전체하차건수_min,전체하차건수_mean,전체하차건수_median,하차건수_max,하차건수_min,하차건수_mean,하차건수_median,미태그건수_max,미태그건수_min,미태그건수_mean,미태그건수_median,노선순서_max,노선순서_min,노선순서_mean,노선순서_median,주중배차간격(분)_max,주중배차간격(분)_min,주중배차간격(분)_mean,주중배차간격(분)_median,주말배차간격(분)_max,주말배차간격(분)_min,주말배차간격(분)_mean,주말배차간격(분)_median,주중상행배차횟수_max,주중상행배차횟수_min,주중상행배차횟수_mean,주중상행배차횟수_median,주중하행배차횟수_max,주중하행배차횟수_min,주중하행배차횟수_mean,주중하행배차횟수_median,주말상행배차횟수_max,주말상행배차횟수_min,주말상행배차횟수_mean,주말상행배차횟수_median,주말하행배차횟수_max,주말하행배차횟수_min,주말하행배차횟수_mean,주말하행배차횟수_median
0,200000006,497,0,91.183938,48.000000,471,0,84.721934,44.000000,116,0,6.462003,3.000000,432,0,42.409758,1.000000,432,0,42.409758,1.000000,0,0,0,0,55.000000,1.000000,11.929688,2.000000,210.000000,6.000000,79.955743,11.500000,210.000000,6.000000,81.545769,17.500000,160.000000,3.833333,73.886349,87.826087,175.000000,3.833333,76.561834,89.565217,160.000000,3.833333,62.330032,57.714286,175.000000,3.833333,64.896849,58.857143
1,200000008,206,8,77.498630,69.000000,206,8,77.200000,69.000000,6,0,0.298630,0.000000,164,0,36.594521,21.000000,164,0,36.594521,21.000000,0,0,0,0,11.000000,11.000000,11.000000,11.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,160.000000,160.000000,160.000000,160.000000,175.000000,175.000000,175.000000,175.000000,160.000000,160.000000,160.000000,160.000000,175.000000,175.000000,175.000000,175.000000
2,200000021,146,14,77.862259,80.000000,136,11,72.944904,74.000000,16,0,4.917355,5.000000,127,10,65.815427,66.000000,127,10,65.815427,66.000000,0,0,0,0,67.000000,67.000000,67.000000,67.000000,8.000000,8.000000,8.000000,8.000000,10.500000,10.500000,10.500000,10.500000,133.750000,133.750000,133.750000,133.750000,133.750000,133.750000,133.750000,133.750000,101.904762,101.904762,101.904762,101.904762,101.904762,101.904762,101.904762,101.904762
3,200000036,133,15,73.186301,74.000000,119,12,66.775342,68.000000,17,0,6.410959,6.000000,174,28,99.109589,102.000000,174,28,99.109589,102.000000,0,0,0,0,29.000000,29.000000,29.000000,29.000000,8.000000,8.000000,8.000000,8.000000,10.500000,10.500000,10.500000,10.500000,133.750000,133.750000,133.750000,133.750000,133.750000,133.750000,133.750000,133.750000,101.904762,101.904762,101.904762,101.904762,101.904762,101.904762,101.904762,101.904762
4,200000047,152,0,24.285463,1.000000,146,0,23.975381,1.000000,8,0,0.310082,0.000000,165,0,24.930832,5.000000,165,0,24.930832,5.000000,0,0,0,0,117.000000,41.000000,72.562500,56.000000,210.000000,13.000000,116.211899,135.000000,210.000000,24.000000,119.100234,135.000000,82.307692,3.833333,27.346116,7.481481,83.846154,3.833333,27.997867,7.481481,44.583333,3.833333,18.781229,7.481481,45.416667,3.833333,19.258605,7.481481


### (4) 첫차 / 막차 시간 

In [241]:
# 첫차 : 가장 빠른 시간 , 막차 : 가장 늦은 시간 
time1 = final2.sort_values('상행첫차').groupby('정류소ID').first().reset_index()
time2 = final2.sort_values('상행막차').groupby('정류소ID').last().reset_index()
time3 = final2.sort_values('하행첫차').groupby('정류소ID').first().reset_index()
time4 = final2.sort_values('하행막차').groupby('정류소ID').last().reset_index()

In [242]:
time1.shape, time2.shape, time3.shape, time4.shape

((516, 33), (516, 33), (516, 33), (516, 33))

# Data Merge 

In [203]:
tmp.shape

(518, 16)

In [204]:
tmp.head(1)

,정류장ID,정류장명,정류장유형,환승역타입,위치(동),쉘터,BIS설치여부,LED,LCD,LED+LCD복합형,알뜰형,임대형(음성),lon,lat,운행노선,중앙차로여부
0,201000345,남수원초등학교,시내,일반,경기도 수원시 세류2동,0.000000,1,0.000000,0.000000,0.000000,1.000000,0,127.016000,37.251091,82-1(수원여객),노변정류장


In [263]:
df = tmp.merge(temp, left_on = '정류장ID', right_on = '정류소ID', copy = False)
del df["정류소ID"]

In [264]:
df = df.rename(columns = {'정류장ID' : '정류소ID'})

In [266]:
df = df.merge(time1[["정류소ID", '상행첫차']], on = '정류소ID', how='left')
df = df.merge(time2[["정류소ID", '상행막차']], on = '정류소ID', how='left')
df = df.merge(time3[["정류소ID", '하행첫차']], on = '정류소ID', how='left')
df = df.merge(time4[["정류소ID", '하행막차']], on = '정류소ID', how='left')

In [268]:
# 중앙차로여부 
df["중앙차로여부"] = df["중앙차로여부"].progress_apply(lambda x : 1 if x == '중앙차로 정류장' else 0)

# 운행노선 / 정류장유형 
df["운행노선"] = df["운행노선"].progress_apply(lambda x : x.split(','))
df["운행노선수"] = df["운행노선"].progress_apply(lambda x : len(x))

df["정류장유형"] = df["정류장유형"].progress_apply(lambda x : x.split(','))
df["정류장유형수"] = df["정류장유형"].progress_apply(lambda x : len(x))

100%|██████████| 516/516 [00:00<00:00, 402578.29it/s]


In [269]:
df.head()

,정류소ID,정류장명,정류장유형,환승역타입,위치(동),쉘터,BIS설치여부,LED,LCD,LED+LCD복합형,알뜰형,임대형(음성),lon,lat,운행노선,중앙차로여부,전체승차건수_max,전체승차건수_min,전체승차건수_mean,전체승차건수_median,초승건수_max,초승건수_min,초승건수_mean,초승건수_median,환승건수_max,환승건수_min,환승건수_mean,환승건수_median,전체하차건수_max,전체하차건수_min,전체하차건수_mean,전체하차건수_median,하차건수_max,하차건수_min,하차건수_mean,하차건수_median,미태그건수_max,미태그건수_min,미태그건수_mean,미태그건수_median,노선순서_max,노선순서_min,노선순서_mean,노선순서_median,주중배차간격(분)_max,주중배차간격(분)_min,주중배차간격(분)_mean,주중배차간격(분)_median,주말배차간격(분)_max,주말배차간격(분)_min,주말배차간격(분)_mean,주말배차간격(분)_median,주중상행배차횟수_max,주중상행배차횟수_min,주중상행배차횟수_mean,주중상행배차횟수_median,주중하행배차횟수_max,주중하행배차횟수_min,주중하행배차횟수_mean,주중하행배차횟수_median,주말상행배차횟수_max,주말상행배차횟수_min,주말상행배차횟수_mean,주말상행배차횟수_median,주말하행배차횟수_max,주말하행배차횟수_min,주말하행배차횟수_mean,주말하행배차횟수_median,상행첫차,상행막차,하행첫차,하행막차,운행노선수,정류장유형수
0,201000345,남수원초등학교,[시내],일반,경기도 수원시 세류2동,0.000000,1,0.000000,0.000000,0.000000,1.000000,0,127.016000,37.251091,[82-1(수원여객)],0,35,1,15.115068,15.000000,31,0,14.271233,14.000000,5,0,0.843836,1.000000,25,1,9.517808,9.000000,25,1,9.517808,9.000000,0,0,0,0,16.000000,16.000000,16.000000,16.000000,10.000000,10.000000,10.000000,10.000000,14.000000,14.000000,14.000000,14.000000,108.500000,108.500000,108.500000,108.500000,107.500000,107.500000,107.500000,107.500000,77.500000,77.500000,77.500000,77.500000,76.785714,76.785714,76.785714,76.785714,1900-01-01 05:00:00,1900-01-01 23:05:00,1900-01-01 06:00:00,1900-01-01 23:55:00,1,1
1,201000268,곡반중학교,[시내],일반,경기도 수원시 곡선동,1.000000,1,1.000000,0.000000,0.000000,0.000000,0,127.034254,37.246081,"[13-5(용남고속), 15(제부여객), 150(제부여객), 151(제부여객), 3...",0,188,0,45.131962,25.000000,145,0,32.542405,19.000000,60,0,12.589557,7.000000,121,0,26.819937,13.000000,121,0,26.819937,13.000000,0,0,0,0,155.000000,16.000000,76.500000,76.000000,80.000000,8.000000,31.659810,21.500000,80.000000,10.500000,35.312342,26.000000,132.500000,11.625000,65.941771,47.441860,133.125000,11.625000,66.902180,47.674419,100.952381,11.625000,51.619876,39.230769,101.428571,11.625000,52.315554,39.423077,1900-01-01 04:10:00,1900-01-01 23:30:00,1900-01-01 04:50:00,1900-01-01 23:50:00,14,1
2,201000357,수원은혜교회.한양수자인파크원아파트,[시내],일반,경기도 수원시 금곡동,1.000000,1,0.000000,1.000000,0.000000,0.000000,0,126.953978,37.266016,"[11-1(수원여객), 123A(용남고속), 123D(용남고속), 13-4(수원여객...",0,166,0,24.081479,11.000000,157,0,22.575845,10.000000,15,0,1.505634,1.000000,49,0,6.957816,4.000000,49,0,6.957816,4.000000,0,0,0,0,91.000000,7.000000,32.187500,8.000000,55.000000,7.000000,26.927767,17.500000,90.000000,9.000000,34.423144,24.500000,147.142857,16.909091,60.950162,58.285714,147.857143,16.909091,60.706798,58.285714,114.444444,10.333333,49.107933,42.448980,115.000000,10.333333,48.900035,42.448980,1900-01-01 04:40:00,1900-01-01 23:00:00,1900-01-01 05:10:00,1900-01-01 23:40:00,15,1
3,202000090,화서역,"[시내, 마을]",일반,경기도 수원시 화서2동,1.000000,1,0.000000,0.000000,1.000000,0.000000,0,126.990107,37.283895,"[16(수원여객), 16-1(수원여객), 16-2(수원여객), 30(수원여객), 3...",0,344,0,23.879602,5.000000,263,0,17.750249,4.000000,81,0,6.129353,2.000000,298,0,43.489552,17.000000,298,0,43.489552,17.000000,0,0,0,0,16.000000,12.000000,14.710938,15.000000,210.000000,10.500000,111.462687,135.000000,210.000000,13.500000,113.121891,135.000000,102.857143,3.833333,41.018697,7.481481,102.857143,3.833333,41.011898,7.481481,80.000000,3.833333,33.187750,7.481481,80.000000,3.833333,33.180951,7.481481,1900-01-01 04:50:00,1900-01-01 23:00:00,1900-01-01 05:40:00,1900-01-01 23:50:00,7,2
4,202000092,월드메르디앙,[시내],일반,경기도 수원시 우만2동,0.000000,1,0.000000,0.000000,1.000000,0.000000,0,127.037045,37.280164,"[3002(용남고속), 7002((주)용남고속버스라인)]",0,11,0,2.357252,2.000000,11,0,2.282443,2.000000,2,0,0.074809,0.000000,96,0,20.247328,12.000000,96,0,20.247328,12.000000,0,0,0,0,63.000000,35.000000,50.593750,63.000000,35.000000,30.000000,32.213740,30.000000,35.000000,30.000000,32.213740,30.000000,36.166667,30.285714,33.562886,36.166667,36.000000,30.285714,33.470011,36.000000,36.166667,30.285714,33.562886,36.166667,36.000000,30.285714,33.470011,36.000000,1900-01-01 05:20:00,1900-01-01 23:30:00,1900-01-01 06:2

In [273]:
df.isnull().sum()

정류소ID               0
정류장명                0
정류장유형               0
환승역타입               0
위치(동)               0
쉘터                  0
BIS설치여부             0
LED                 0
LCD                 0
LED+LCD복합형          0
알뜰형                 0
임대형(음성)             0
lon                 0
lat                 0
운행노선                0
중앙차로여부              0
전체승차건수_max          0
전체승차건수_min          0
전체승차건수_mean         0
전체승차건수_median       0
초승건수_max            0
초승건수_min            0
초승건수_mean           0
초승건수_median         0
환승건수_max            0
환승건수_min            0
환승건수_mean           0
환승건수_median         0
전체하차건수_max          0
전체하차건수_min          0
전체하차건수_mean         0
전체하차건수_median       0
하차건수_max            0
하차건수_min            0
하차건수_mean           0
하차건수_median         0
미태그건수_max           0
미태그건수_min           0
미태그건수_mean          0
미태그건수_median        0
노선순서_max            0
노선순서_min            0
노선순서_mean           0
노선순서_median         0
주중배차간격(분)_max       0
주중배차간격(분)_

In [270]:
df.to_excel(current_path+'/busdata/busdf_0304.xlsx', index=False)